# About:

In [1]:
%reload_ext autoreload
%autoreload 2

import os, sys

sys.path.append(os.path.abspath(os.pardir))

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
COUNTRY_NAME = os.environ.get("COUNTRY_NAME")
SPOKEN_LANGUAGE = "French"

table_column_name = "TexteCourt"

In [5]:
from general_config import COUNTRY_NAMES_LIST

# validate COUNTRY_NAME
if COUNTRY_NAME in COUNTRY_NAMES_LIST:
    print(COUNTRY_NAME, 'country name OK')

MLI Mali country name OK


In [6]:
from llm_setup import *

# Get all documents for the selected country

In [8]:
(df, documents) = get_country_legal_docs(COUNTRY_NAME)

/home/andrei/Ferdi_LLM/postgres_connection.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


# Get all documents for the selected country

In [9]:
embedding_model = get_voyage_ai_embedding_model()

vector_index = get_vector_db_from_documents(documents, COUNTRY_NAME, embedding_model, chunk_size=1024)

loading from local


In [10]:
doc_titles = pd.Series(list(set([v.get('title') for k,v in vector_index.vector_store.to_dict()['metadata_dict'].items()])))
doc_titles[0]

'img_Ordonnance n°1999-032 (19.08.1999) Code minier 1999 (Ministère des Mines)'

# Set the llm
the llm will be used to augment the retrieved data from the vector_index

In [16]:
llm = get_open_ai_llm()

# Get TexteCourt

In [17]:
from difflib import get_close_matches

def find_most_similar_string(target, string_list):
    # Using difflib's get_close_matches function to find the closest match
    # It returns a list of close matches; we take the first (most similar) one
    closest_matches = get_close_matches(target, string_list, n=1, cutoff=0.0)
    return closest_matches[0] if closest_matches else None

def get_texte_court(vector_index, doc_name, llm):
    vector_query_engine = create_vector_engine(vector_index, doc_name, llm)

    """
    You are a helpful assistant, with legal document analysis expertise. Please analyze the given context.
    """
    response = vector_query_engine.query(
        f"""
        Consider the contents of the document with this title '{doc_name}'.
        Generate a short phrase, in French, that describes the document, containing the year of the document and the year of the original document.
        Here are some samples: Code général des impôts 1970, Modification 1971 du Code général des impôts 1970, Modification 1973 du Code général des impôts 1970, Modification 1985 du Code général des impôts 1970 
        If you can't tell exactly to which document it's related, then do not add that information.
        """
    )

    print(doc_name, len(response.source_nodes))
    print([node.score for node in response.source_nodes])
    print([node.text for node in response.source_nodes])

    return response.response

In [18]:
response = get_texte_court(
    vector_index=vector_index,
    doc_name = doc_titles[0],
    llm=llm)

print(response)

img_Ordonnance n°1999-032 (19.08.1999) Code minier 1999 (Ministère des Mines) 2
[0.88671875, 0.8828125]
["006837\n\nPRIMATURE\nSECRETARIAT GENERAL DU GOUVERNEMENT\n\nREPUBLIQUE DU MALI\nUn Peuple - Un But - Une Foi\n\nORDONNANCE N° 99-032/P-RM DU 19 AOUT 1999\n\nPORTANT CODE MINIER EN REPUBLIQUE DU MALI\n\nLE PRESIDENT DE LA REPUBLIQUE,\n\nVU la Constitution ;\nVU la Loi N°99-034 du 04 juillet 1999 autorisant le Gouvernement à prendre certaines mesures par ordonnance ;\nVU le Décret N°97-263/P-RM du 13 septembre 1997 portant nomination d’un Premier ministre ;\nVU le Décret N°97-282/P-RM du 16 septembre 1997 portant nomination des membres du Gouvernement ;\n\nLa Cour Suprême entendue;\n\nSTATUANT EN CONSEIL DES MINISTRES,\n\nORDONNE : **SOMMAIRE**\n\n**TITRE I** - **DES DISPOSITION GENERALES** 2 à 10\n  - **Chapitre I** - Des Définitions\n  - **Chapitre II** - De la Classification des gîtes de substances minérales ou fossiles\n  - **Chapitre III** - De la Convention d'Etablissement\n\n*

In [19]:
texte_courts = list()

for doc in doc_titles:
    try:
        categ = get_texte_court(
            vector_index=vector_index,
            doc_name = doc,
            llm=llm
            )
    except Exception as e:
        print(e)

        categ = "non_determined"

    texte_courts.append((doc, categ))

    

img_Ordonnance n°1999-032 (19.08.1999) Code minier 1999 (Ministère des Mines) 2
[0.88671875, 0.8828125]
["006837\n\nPRIMATURE\nSECRETARIAT GENERAL DU GOUVERNEMENT\n\nREPUBLIQUE DU MALI\nUn Peuple - Un But - Une Foi\n\nORDONNANCE N° 99-032/P-RM DU 19 AOUT 1999\n\nPORTANT CODE MINIER EN REPUBLIQUE DU MALI\n\nLE PRESIDENT DE LA REPUBLIQUE,\n\nVU la Constitution ;\nVU la Loi N°99-034 du 04 juillet 1999 autorisant le Gouvernement à prendre certaines mesures par ordonnance ;\nVU le Décret N°97-263/P-RM du 13 septembre 1997 portant nomination d’un Premier ministre ;\nVU le Décret N°97-282/P-RM du 16 septembre 1997 portant nomination des membres du Gouvernement ;\n\nLa Cour Suprême entendue;\n\nSTATUANT EN CONSEIL DES MINISTRES,\n\nORDONNE : **SOMMAIRE**\n\n**TITRE I** - **DES DISPOSITION GENERALES** 2 à 10\n  - **Chapitre I** - Des Définitions\n  - **Chapitre II** - De la Classification des gîtes de substances minérales ou fossiles\n  - **Chapitre III** - De la Convention d'Etablissement\n\n*

In [20]:
df_TexteFiscExactCode = pd.DataFrame.from_dict(dict(texte_courts), orient='index', columns=['TexteCourt'])

df_TexteFiscExactCode.to_csv(f"output/TexteCourt_{COUNTRY_NAME}.csv", index=True)

df_TexteFiscExactCode

,TexteCourt
img_Ordonnance n°1999-032 (19.08.1999) Code minier 1999 (Ministère des Mines),"Code minier 1999, Ordonnance n°91-065/P-CTSP d..."
img_Décret n°1999-255 (15.09.1999) Application Code minier 1999 (Ministère des Mines),"Application du Code minier 1999, Décret n°1999..."
Loi n°1996-021 (21.02.1966) Jeux de hasard 1996 (SGG),Loi n°1996-021 sur les jeux de hasard 1996.
img_Loi n°2012-015 (27.02.2012) Code minier 2012 (Ministère des Mines),"Code minier 2012, Modification 1999 du Code mi..."
img_Ordonnance n°2000-013 (10.02.2000) Modification Code minier 2000 (Ministère des Mines),Modification 2000 du Code minier 1999
...,...
Rappel historique des réformes fiscales au Mali,Rappel historique des réformes fiscales au Mal...
img_Ordonnance n°1991-065 (19.09.1991) Code minier 1991 (Ministère des Mines),"Code minier 1991, Modification 1981 du Code mi..."
Contrat de partage de production type (Ministère des Mines),Contrat de partage de production type 2023
Loi n°2019-070 (24.12.2019) Loi de finances 2020 (Ministère des Finances),"Loi de finances 2020, Loi n°2019-070 de 2019"


## Load existing categories

In [24]:
COUNTRY_NAME = 'BEN Bénin'

large_categs = pd.read_csv(f"output/{table_column_name}_{COUNTRY_NAME}.csv")
large_categs = list(list(large_categs.set_index(['Unnamed: 0']).to_dict().values())[0].items())

## Update column in table

In [25]:
from postgres_connection import bulk_update_textes

In [26]:
# bulk_update_employees(textes_to_update = large_categs[0:1], )

bulk_update_textes(
    textes_to_update = large_categs, 
    column_name = table_column_name, 
    COUNTRY_NAME=COUNTRY_NAME)

Ordonnance n°1966-002 (10.01.1966) Code général des impôts 2010 (eRegularisation Bénin) déverrouillé column TexteCourt updated to Code général des impôts 2010, Ordonnance n°1966-002 de 1966.
img_Loi n°2005-042 (27.12.2006) Loi de finances 2006 (LégiBénin) column TexteCourt updated to Loi de finances 2006, Modification 2005 du Code général des impôts..
img_Loi n°2007-033 (02.01.2008) Loi de finances 2008 (LégiBénin) column TexteCourt updated to Loi de finances 2008, Loi de finances 2007.
Loi n°2005-016 (08.09.2005) Zone franche industrielle 2005 (Droit-Afrique) column TexteCourt updated to Loi sur la zone franche industrielle 2005, Loi n° 99-001 de 1999.
Liste des lois (Assemblée Nationale) column TexteCourt updated to Liste des lois 2006, Compilation des lois votées et promulguées 1990.
img_Loi n°2006-024 (26.12.2006) Loi de finances 2007 (LégiBénin) column TexteCourt updated to Loi de finances 2007, Modification 2006 de la Loi de finances 2007.
img_Loi n°1997-001 (21.01.1997) Loi de f